In [1]:
#### importing dependencies

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Model, load_model, Sequential, model_from_yaml # to save our weights and load the pre-trained weights model
from keras.layers import Dense, Activation, Dropout, Input, Masking
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import random
import sys
import io

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#### Importing the dataset

def read_data(path):
    print("Loading text data...")
    text = io.open(path, encoding='utf-8').read().lower()
    print("Done.")
    print("Corpus Length:", len(text))
    
    return text

In [16]:
text=read_data('ramayana.txt')

Loading text data...
Done.
Corpus Length: 1967249


In [17]:
#### Calculating the number of characters stored and building the c_to_i dictionary

chars=sorted(list(set(text)))

print('number of unique characters in the corpus:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Tx = 40 # sequence length

number of unique characters in the corpus: 83


In [18]:
#### Helper functions to build (train examples & labels) the dataset & vectorize it

def build_data(text, Tx = 40, steps = 3):
    """Create a training set by scanning a window of size Tx over the text corpus, with a stride of 3."""
    X = [] # training examples (list)
    Y = [] # training labels (list)
    for i in range(0, len(text) - Tx, steps):
        X.append(text[i: i + Tx])
        Y.append(text[i + Tx])
    print('number of training examples:', len(X))
    return X, Y


def vectorization(X, Y, n_x, char_indices, Tx = 40):
    """Convert X and Y (lists) into arrays to be given to a recurrent neural network."""
    
    m = len(X)
    x = np.zeros((m, Tx, n_x), dtype=np.bool) # array of shape (m, Tx, len(chars))
    y = np.zeros((m, n_x), dtype=np.bool) # array of shape (m, len(chars))
    for i, sentence in enumerate(X):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[Y[i]]] = 1
        
    return x, y 


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    out = np.random.choice(range(len(chars)), p = probas.ravel())
    return out
    

In [19]:
#### Building the data and Vectorizing it

X, Y = build_data(text, Tx=40, steps=3)
x, y = vectorization(X, Y, n_x = len(chars), char_indices = char_indices)


number of training examples: 655737


In [20]:
#### Defining the Model

print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(Tx, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [21]:
#### Function invoked at end of each epoch. Prints generated text.

def on_epoch_end(epoch, logs, maxlen=Tx):
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    diversity=0.4

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    #usr_input = input("Write the beginning sentence and machine will complete it. Your input is: ")
    # zero pad the sentence to Tx characters.
    #sentence = ('{0:0>' + str(Tx) + '}').format(usr_input).lower()
    #generated += usr_input
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(1000):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
        
        if next_char == '\n':
            continue

In [22]:
#### Generating the output

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=1,
          callbacks=[print_callback])

Epoch 1/1
655616/655737 [============================>.] - ETA: 0s - loss: 1.7088
----- Generating text after Epoch: 0
----- Generating with seed: "shore,
and, deeming that all hope was lo"
shore,
and, deeming that all hope was lord,
the sangless and the monarch steed,
with the will lord and realm the soul,
and there the good in speatles to see,
and brave the king the sangure stay,
the stream in the his angues to the giant speech
and for the son in sangs the stand
of all the consolled the strong.
then with the brothers to the soul.
and still the consort on the sangle,
the words the spite of her the speech and pride
and world the streef the world, and speech.
the souther of the saint to see,
and can i see the words the near,
and standed the comman dread.
the giant still the lord of the be
the stranger to the words to streed.
the sage and forth the trees to see,
and holy roins with sent the side,
where soul the mighty soul the be
the words the words the world to streed.
the path the sang

In [10]:
# Saving weights and model
"""
model_yaml = model.to_yaml()
with open("ramayana_model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

model.save_weights("ramayana_model.h5")
print("Saved model to disk")
"""

'\nmodel_yaml = model.to_yaml()\nwith open("ramayana_model.yaml", "w") as yaml_file:\n    yaml_file.write(model_yaml)\n\nmodel.save_weights("ramayana_model.h5")\nprint("Saved model to disk")\n'